In [7]:
!pip install sentence_transformers
!pip install pytrec_eval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [8]:
from create_faiss_index import create_and_store_dense_index_approx
import pandas as pd
from run_query_faiss import query_dense_index
from sentence_transformers import SentenceTransformer
from dataloader import get_data
from tqdm import tqdm
import pytrec_eval
import os
import json

In [18]:
# # Example Usage
# # documents = [
# #     "The quick brown fox jumps over the lazy dog.",
# #     "Sentence embeddings are useful for dense retrieval.",
# #     "LaBSE is a multilingual model for sentence embeddings."
# # ]

tasks = ["monolingual","crosslingual"]
splits = ["train", "dev"]
models = [
    ["sentence-transformers/msmarco-bert-co-condensor"],["facebook-dpr-ctx_encoder-single-nq-base","facebook-dpr-question_encoder-single-nq-base"],
    ["sentence-transformers/distiluse-base-multilingual-cased-v2"],[""]
]
model_names = ["bert_codensor","fb_dpr" , "distiluse_multilingual_v2"]
model_name = "sentence-transformers/msmarco-bert-co-condensor"
model_name_short = "distiluse_multilingual_v2"

bert_multilingual = "amberoad/bert-multilingual-passage-reranking-msmarco"

model_name = bert_multilingual#  "sentence-transformers/distiluse-base-multilingual-cased-v2"
model_name_query = bert_multilingual #"sentence-transformers/msmarco-bert-co-condensor"
# model_name_short = "distiluse_multilingual_v2"
model_name_short_query = f"{model_name_short}_query"




create_and_store_index = True



TASK = "monolingual"
SPLIT = "train"
# Get documents as a list of fact-checks from docs_monolingual.csv
doc_csv_path = "final_expanded_docs_monolingual_train.csv" #"final_expanded_docs_crosslingual_dev.csv" # f"docs_{TASK}_{SPLIT}.csv"
query_csv_path = f"queries_{TASK}_{SPLIT}.csv"
df_documents = pd.read_csv(doc_csv_path)
df_documents['ext_doc'] = df_documents['doc'] + " " + df_documents['expanded_doc']
documents = df_documents['ext_doc'].astype(str).tolist()

output_directory = f"{TASK}_{SPLIT}_dense_index_{model_name_short}_ivf"
FINAL_OUTPUT_DIR = f'{model_name_short}_{TASK}_{SPLIT}'

device = "cuda"

# Index path 
index_path = f"{output_directory}/faiss_index_ivf"
metadata_path = f"{output_directory}/metadata.pkl"

## save results
if not os.path.exists(FINAL_OUTPUT_DIR):
    os.makedirs(FINAL_OUTPUT_DIR)


In [19]:
create_and_store_dense_index_approx(documents, model_name, output_directory)


No sentence-transformers model found with name amberoad/bert-multilingual-passage-reranking-msmarco. Creating a new one with mean pooling.


Encoding documents...


Batches:   0%|          | 0/2680 [00:00<?, ?it/s]

Creating FAISS IndexIVFFlat...
Training FAISS index...
Saving FAISS index to monolingual_train_dense_index_distiluse_multilingual_v2_ivf/faiss_index_ivf...
Saving metadata to monolingual_train_dense_index_distiluse_multilingual_v2_ivf/metadata.pkl...
Index and metadata successfully saved.


In [20]:

# Load the model
model = SentenceTransformer(model_name_query)
model = model.to(device)
    


# Document csv
df_docs = pd.read_csv(doc_csv_path)
doc_ids = df_docs['doc_ids'].tolist()

# Query csv 
df_queries = pd.read_csv(query_csv_path)
query_ids = df_queries['query_ids'].tolist()
 
# df_fact_check_post_mapping
_, _, df_fact_check_post_mapping = get_data('./data')





No sentence-transformers model found with name amberoad/bert-multilingual-passage-reranking-msmarco. Creating a new one with mean pooling.


In [21]:

# Metrics
metrics=['P_3', 'P_5', 'P_10', 'map_cut_3', 'map_cut_5', 'map_cut_10', 'ndcg_cut_3', 'ndcg_cut_5', 'ndcg_cut_10']
average_scores = {metric: 0.0 for metric in metrics}




count = 0

for _, row in tqdm(df_queries.iterrows(), total=len(df_queries)):
        
        qrels = {}
        runs = {}
        
        query = row['query']
        idx = query_ids[count] 
        
        #print("Query:", query)
        #print("Query Id:", idx)
        
        # Results from dense multilingual model 
        results = query_dense_index(query, model, index_path, metadata_path, top_k=10)

        ranked_fact_checks = []
        ranked_fact_checks_scores = []
        
        for rank, (doc, score) in enumerate(results, start=1):
            
            doc_id = doc_ids[doc]
            #print("Doc_id", doc_id)
            ranked_fact_checks.append(doc_id) 
            ranked_fact_checks_scores.append(score)
            
            #runs[str(idx)][str(doc_id)] = score 
            #print(f"Rank {rank}: {doc} (Score: {score})")
        
        
        runs[str(idx)] = {}
        qrels[str(idx)] = {} 
        
        for fc_id, score in zip(ranked_fact_checks, ranked_fact_checks_scores):
            runs[str(idx)][str(fc_id)] = float(score) 
        
        
        ground_truth = df_fact_check_post_mapping[
            df_fact_check_post_mapping['post_id'] == idx
        ]['fact_check_id'].tolist()

        ## keys must be strings
        for fc_id in ground_truth:
            qrels[str(idx)][str(fc_id)] = 1
        
        
        #qrels = {str(k): {str(d): v for d, v in docs.items()} for k, docs in qrels.items()}
        #run = {str(k): {str(d): v for d, v in docs.items()} for k, docs in runs.items()}

        #print("Metrics",metrics)

        evaluator = pytrec_eval.RelevanceEvaluator(qrels, metrics)
        #print("Evaluator", evaluator)
        
        #print("Qrels:", qrels)
        #print("Qrels type", type(qrels))
        #print("Runs:", runs)
        #print("Runs type", type(runs))
        results_scores = evaluator.evaluate(runs)

        #print("Results", results_scores)

        for query_id in results_scores:
            for metric in metrics:
                average_scores[metric] += results_scores[query_id][metric]

        with open(f"{FINAL_OUTPUT_DIR}/{model_name_short}_{TASK}_intermediate_scores_{count}.json", 'w') as f1:
            json.dump(average_scores, f1, indent=1)

        count = count + 1 
        # if count == 5:
        #     break

        

for metric in average_scores:
    average_scores[metric] /= count

print("Average scores:")
print(average_scores)



with open(f"{FINAL_OUTPUT_DIR}/{model_name_short}_{TASK}_finalresults.json", 'w') as f:
    json.dump(average_scores, f, indent=1)



100%|██████████| 4351/4351 [08:22<00:00,  8.66it/s]

Average scores:
{'P_3': 0.006358691488546687, 'P_5': 0.0056079062284532175, 'P_10': 0.004091013560101123, 'map_cut_3': 0.010320743634924284, 'map_cut_5': 0.011938187900610336, 'map_cut_10': 0.013327243995481195, 'ndcg_cut_3': 0.01220544045343359, 'ndcg_cut_5': 0.015139086499540934, 'ndcg_cut_10': 0.018590973758298695}


In [15]:
# some new models to try: ce rerankers  -for later
bert_multilingual = "amberoad/bert-multilingual-passage-reranking-msmarco"

# few more models for retriever:
distilbert = "msmarco-distilbert-base-v4"
MiniLM = "msmarco-MiniLM-L-12-v3"

'''

Average scores monolingual model: fb_dpr:
{'P_3': 0.06335708266298971, 'P_5': 0.04716157205240113, 'P_10': 0.02953344058836992, 'map_cut_3': 0.1185736356903903, 'map_cut_5': 0.12758286728465987, 'map_cut_10': 0.13480108872375013, 'ndcg_cut_3': 0.13317919711870022, 'ndcg_cut_5': 0.1482876721102965, 'ndcg_cut_10': 0.16442949118297137}

Average scores crosslingual model: fb_dpr:
{'P_3': 0.012939125770984206, 'P_5': 0.010016090104585708, 'P_10': 0.0061142397425583556, 'map_cut_3': 0.025936354697416626, 'map_cut_5': 0.028473898274783218, 'map_cut_10': 0.029756716980168303, 'ndcg_cut_3': 0.028708956783380736, 'ndcg_cut_5': 0.03309607110639242, 'ndcg_cut_10': 0.03622373972360996}


Average scores crosslingual model: distiluse_multilingual_v2:
{'P_3': 0.08400375435773544, 'P_5': 0.061584875301687984, 'P_10': 0.03757039420756129, 'map_cut_3': 0.16668398587646407, 'map_cut_5': 0.17808902073835772, 'map_cut_10': 0.1864882255977392, 'ndcg_cut_3': 0.18404025900939375, 'ndcg_cut_5': 0.20366602117504182, 'ndcg_cut_10': 0.2225580157383061}

Average scores monolingual model: distiluse_multilingual_v2:
{'P_3': 0.14241936719528095, 'P_5': 0.10678005056308547, 'P_10': 0.06393932429326495, 'map_cut_3': 0.2605895196506553, 'map_cut_5': 0.2820931969662154, 'map_cut_10': 0.29533224436637456, 'ndcg_cut_3': 0.29221580694965205, 'ndcg_cut_5': 0.325354511127512, 'ndcg_cut_10': 0.35229995882726817}


Average scores Expanded doc: crosslingual model: distiluse_multilingual_v2:
{'P_3': 0.07978010190399466, 'P_5': 0.05812550281576693, 'P_10': 0.03608205953338594, 'map_cut_3': 0.1589551488334677, 'map_cut_5': 0.16952969965138173, 'map_cut_10': 0.17795163763637245, 'ndcg_cut_3': 0.17541911996046922, 'ndcg_cut_5': 0.19361040023418608, 'ndcg_cut_10': 0.2130294812501804}

Expanded doc: cross lingual "sentence-transformers/msmarco-bert-co-condensor": 
Average scores:
{'P_3': 0.057656208098686185, 'P_5': 0.04030571198712748, 'P_10': 0.023732904263877325, 'map_cut_3': 0.1206489675516223, 'map_cut_5': 0.12630413649772054, 'map_cut_10': 0.13073643089129813, 'ndcg_cut_3': 0.13100057790167163, 'ndcg_cut_5': 0.1403265820846624, 'ndcg_cut_10': 0.15027825575033718}

Expanded doc crosslingual fb_dpr
{'P_3': 0.010257441673370893, 'P_5': 0.007884151246983107, 'P_10': 0.0052695092518101555, 'map_cut_3': 0.020104809153481718, 'map_cut_5': 0.02188142486815053, 'map_cut_10': 0.023484848484848483, 'ndcg_cut_3': 0.02255028219883454, 'ndcg_cut_5': 0.025730097791489014, 'ndcg_cut_10': 0.02950918546899713}
 

Average scores: monolingual expanded doc: fb_dpr
{'P_3': 0.0606757067340846, 'P_5': 0.04541484716157152, 'P_10': 0.02764881636405374, 'map_cut_3': 0.11679945861743132, 'map_cut_5': 0.12558671059015789, 'map_cut_10': 0.13102158598826028, 'ndcg_cut_3': 0.13045171167588318, 'ndcg_cut_5': 0.1454426197168512, 'ndcg_cut_10': 0.15807466967596806}

Average scores: monolingual expanded doc:  bert_codensor

{'P_3': 0.1408105416379377, 'P_5': 0.10613652034014821, 'P_10': 0.06430705584922906, 'map_cut_3': 0.26120942822850446, 'map_cut_5': 0.2828239995914102, 'map_cut_10': 0.2969226316640707, 'ndcg_cut_3': 0.2919568167203163, 'ndcg_cut_5': 0.3262449638643935, 'ndcg_cut_10': 0.3555820013886881}

Average scores: monolingual expanded doc:  distiluse_multilingual_v2
{'P_3': 0.006358691488546687, 'P_5': 0.0056079062284532175, 'P_10': 0.004091013560101123, 'map_cut_3': 0.010320743634924284, 'map_cut_5': 0.011938187900610336, 'map_cut_10': 0.013327243995481195, 'ndcg_cut_3': 0.01220544045343359, 'ndcg_cut_5': 0.015139086499540934, 'ndcg_cut_10': 0.018590973758298695}

'''

'\n\nAverage scores monolingual model: fb_dpr:\n{\'P_3\': 0.06335708266298971, \'P_5\': 0.04716157205240113, \'P_10\': 0.02953344058836992, \'map_cut_3\': 0.1185736356903903, \'map_cut_5\': 0.12758286728465987, \'map_cut_10\': 0.13480108872375013, \'ndcg_cut_3\': 0.13317919711870022, \'ndcg_cut_5\': 0.1482876721102965, \'ndcg_cut_10\': 0.16442949118297137}\n\nAverage scores crosslingual model: fb_dpr:\n{\'P_3\': 0.012939125770984206, \'P_5\': 0.010016090104585708, \'P_10\': 0.0061142397425583556, \'map_cut_3\': 0.025936354697416626, \'map_cut_5\': 0.028473898274783218, \'map_cut_10\': 0.029756716980168303, \'ndcg_cut_3\': 0.028708956783380736, \'ndcg_cut_5\': 0.03309607110639242, \'ndcg_cut_10\': 0.03622373972360996}\n\n\nAverage scores crosslingual model: distiluse_multilingual_v2:\n{\'P_3\': 0.08400375435773544, \'P_5\': 0.061584875301687984, \'P_10\': 0.03757039420756129, \'map_cut_3\': 0.16668398587646407, \'map_cut_5\': 0.17808902073835772, \'map_cut_10\': 0.1864882255977392, \'nd